## Loading data into GCS

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']

'/Users/ns/github-repos/energy-dashboard/energy-dashboard.json'

In [3]:
credentials = service_account.Credentials.from_service_account_file(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
project_id = 'ml-energy-dashboard'

In [11]:
bucket_name = 'ml-energy-dashboard-raw-data'
storage_client = storage.Client(credentials=credentials, project=project_id)

if not storage_client.bucket(bucket_name):
    print(f'Bucket exists: {bucket_name}')
else:
    print(f'Creating bucket: {bucket_name}')
    storage_client.create_bucket(bucket_name)


Creating bucket: ml-energy-dashboard-raw-data


In [12]:
bucket=storage_client.get_bucket(bucket_name)
blob = bucket.blob('es_load_data')


In [15]:
from entsoe import EntsoePandasClient
import pandas as pd

In [16]:
client = EntsoePandasClient(api_key = os.environ['ENTSOE_TOKEN'])
data = client.query_load("ES",
                  start=pd.Timestamp("20200214", tz='UTC'), 
                  end=pd.Timestamp("20200305", tz='UTC'))

In [18]:
!pwd

/Users/ns/github-repos/energy-dashboard/notebooks


In [21]:
path='/Users/ns/github-repos/energy-dashboard/data'
name = 'es_load_data_20200305-16'
#data.to_csv(os.path.join(path, name))

In [22]:
blob.upload_from_filename(os.path.join(path, name))